In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import random
import time
from datetime import datetime
from numpy.random import choice
from faker import Faker
from random import randint

In [2]:
df = pd.DataFrame(columns=['OrderID','Startdate', 'Enddate',
                           'Priority', 'Status', 'ActLaborHrs',
                           'Group', 'SubGroup', 'Responsedays', 'Responsehours'])
df.head()

,OrderID,Startdate,Enddate,Priority,Status,ActLaborHrs,Group,SubGroup,Responsedays,Responsehours


### Generate Orders by Timestamp

#### All generated orders follow business days (Mon-Fri) and hours (8am-6pm)

In [3]:
def randomDate(start, end):
    '''
    input: setup start and end date 
    output: one random date between the range(start, end) and follows business days/ hours
    '''
    from datetime import datetime
    frmt = '%Y-%m-%d %H:%M:%S'
    busday = False 
    start_time = time.mktime(time.strptime(start, frmt))
    end_time = time.mktime(time.strptime(end, frmt))
    
    while busday == False:
        random_time = start_time + random.random() * (end_time - start_time)
        dt = datetime.fromtimestamp(time.mktime(time.localtime(random_time)))
        busday = businessday(dt)
    return dt

In [4]:
def businessday(dt):
    '''
    input: any 'time' data
    output: a boolean value shows if the input follow the business days/ hours
    '''
    if dt.weekday() == 5 or dt.weekday() == 6: #Mon - Fri
        return False
    else:
        if dt.hour <= 8 or dt.hour >= 18: #8am - 6pm
            return False
        else:
            return True

In [5]:
def get_start_date(start, end, amount):
    '''
    This function conclude above two
    input: setup start and end date with form(%Y-%m-%d %H:%M:%S) ; how many random date you need
    output: a list of random dates between the range(start, end) and follow business days/ hours
    '''
    start_date_list = []
    for i in range(0,amount):
        start_date = randomDate(start, end)
        start_date_list.append(str(start_date))
    return start_date_list

In [87]:
def get_end_date(start, duration):
    '''
    input: one or a list of order start date
    output: a list of order end date by random 1 ~ duration
    '''
    frmt = '%Y-%m-%d %H:%M:%S'
    end_date_list = []
    diff_day_list = []
    diff_hour_list = []
    
    for i in range(0, len(start)):
        busday = False
        start_time = time.mktime(time.strptime(start[i], frmt))
        while busday == False:
            random_diff = random.randrange(1, duration, 6) * 3600 #by hours
            new_date = start_time + random_diff
            dt = datetime.fromtimestamp(time.mktime(time.localtime(new_date)))
            busday = businessday(dt)
        end_date_list.append(str(dt))
        diff_day_list.append(str(random_diff/86400))
        diff_hour_list.append(str(random_diff/3600))
        
    return end_date_list, diff_day_list, diff_hour_list

#### Oder Startdate:

In [7]:
random_start = get_start_date('2018-01-01 00:00:00', '2019-12-31 00:00:00', 20000)
df['Startdate'] = random_start

#### Order EndDate, Responsedays, Responsehours:

In [88]:
random_end, random_diffday,random_diffhour  = get_end_date(random_start, 100)

In [95]:
df['Enddate'] = random_end
df['Responsedays'] = random_diffday
df['Responsehours'] = random_diffhour

In [96]:
df.head(5)

,OrderID,Startdate,Enddate,Priority,Status,ActLaborHrs,Group,SubGroup,Responsedays,Responsehours
0,WO156580,2019-04-05 09:31:17,2019-04-08 10:31:17,2.0,Complete,1.000000,HVAC,NaN,3.0416666666666665,73.0
1,WO372550,2018-01-05 13:14:41,2018-01-05 14:14:41,11.0,Complete,9.039537,Engineering,NaN,0.041666666666666664,1.0
2,WO293904,2019-01-31 14:49:44,2019-02-01 09:49:44,2.0,Complete,9.039537,CallCenter,NaN,0.7916666666666666,19.0
3,WO220975,2019-06-04 16:05:03,2019-06-07 17:05:03,11.0,Complete,1.000000,Plumber,NaN,3.0416666666666665,73.0
4,WO352047,2019-08-28 15:55:37,2019-08-28 16:55:37,11.0,Complete,0.500000,Engineering,NaN,0.041666666666666664,1.0


In [10]:
df['Startdate'].tail(5)

19995    2019-12-13 12:42:12
19996    2018-03-30 16:10:03
19997    2019-11-18 10:20:37
19998    2019-05-10 17:27:51
19999    2019-11-13 16:13:34
Name: Startdate, dtype: object

### ActLaborHrs

In [112]:
actlabhour = []
for i in df['Responsehours']:
    actlabhour.append(random.uniform(0, float(i))) # random generate actually labor hours < Responsehours

In [113]:
df['ActLaborHrs'] = actlabhour

In [114]:
df['ActLaborHrs'].describe()

count    20000.000000
mean        21.990993
std         23.036440
min          0.000019
25%          2.382022
50%         14.636911
75%         34.585183
max         96.987639
Name: ActLaborHrs, dtype: float64

In [115]:
df['ActLaborHrs'].head(5)

0    65.612659
1     0.708506
2    17.363267
3    24.791139
4     0.217532
Name: ActLaborHrs, dtype: float64

### Priority

In [11]:
df['Priority'] = np.random.choice([1, 2, 11]) #df[0]

In [12]:
df['Priority'].value_counts()

2    20000
Name: Priority, dtype: int64

In [36]:
dfupdate = df[df['Priority'] == 2 ].sample(2973)

In [37]:
dfupdate['Priority'] = 11
df.update(dfupdate)

In [38]:
df['Priority'].value_counts()

11.0    14865
2.0      5135
Name: Priority, dtype: int64

### OrderID

In [47]:
df['OrderID'] = random.sample(range(100000,700000), 20000)

In [49]:
df['OrderID'].count()

20000

In [50]:
dfN['OrderID'].duplicated().sum()

0

In [51]:
dfN['OrderID'] = 'WO' + dfN['OrderID'].astype(str)

In [52]:
dfN['OrderID'].head(20)

0     WO156580
1     WO372550
2     WO293904
3     WO220975
4     WO352047
5     WO204383
6     WO348957
7     WO389943
8     WO506187
9     WO415139
10    WO535841
11    WO206977
12    WO292917
13    WO104085
14    WO623504
15    WO655863
16    WO496377
17    WO280863
18    WO149296
19    WO473759
Name: OrderID, dtype: object

### Group&SubGroup

In [54]:
def get_employee_names(people):
    l = []
    for i in range(0, people):
        n = fake.name()
        l.append(n)
    return l

In [55]:
group_list = ['EHS', 'Engineering', 'HVAC', 'Electrical',
              'Fire safety', 'Plumber', 'Locksmith', 'CallCenter', 'Landscape service',
              'Security', 'Cleaning', 'Energy', 'Building Maintenance',
              'IT', 'Managment']
group_weight = [0.005, 0.15, 0.13, 0.03,
                0.09, 0.08, 0.10, 0.08, 0.003,
                0.03, 0.09, 0.07, 0.133,
                0.005, 0.004]

In [56]:
sum(group_weight)

1.0

In [57]:
len(group_list)

15

In [58]:
df['Group'] =  choice(group_list, 20000, p = group_weight)

In [59]:
df['Group'].value_counts()

Engineering             2989
Building Maintenance    2699
HVAC                    2624
Locksmith               2021
Fire safety             1795
Cleaning                1715
Plumber                 1640
CallCenter              1567
Energy                  1408
Electrical               594
Security                 591
EHS                      108
IT                       103
Managment                 86
Landscape service         60
Name: Group, dtype: int64

In [61]:
df

,OrderID,Startdate,Enddate,Priority,Status,ActLaborHrs,Group,SubGroup,Responsedays,Responsehours
0,WO156580,2019-04-05 09:31:17,2019-04-05 16:31:17,2.0,NaN,1.000000,HVAC,NaN,NaN,NaN
1,WO372550,2018-01-05 13:14:41,2018-01-09 14:14:41,11.0,NaN,9.039537,Engineering,NaN,NaN,NaN
2,WO293904,2019-01-31 14:49:44,2019-02-01 09:49:44,2.0,NaN,9.039537,CallCenter,NaN,NaN,NaN
3,WO220975,2019-06-04 16:05:03,2019-06-07 11:05:03,11.0,NaN,1.000000,Plumber,NaN,NaN,NaN
4,WO352047,2019-08-28 15:55:37,2019-08-28 16:55:37,11.0,NaN,0.500000,Engineering,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
19995,WO372402,2019-12-13 12:42:12,2019-12-13 13:42:12,2.0,NaN,1.000000,Building Maintenance,NaN,NaN,NaN
19996,WO371224,2018-03-30 16:10:03,2018-04-02 17:10:03,2.0,NaN,9.039537,Electrical,NaN,NaN,NaN
19997,WO436980,2019-11-18 10:20:37,2019-11-21 17:20:37,11.0,NaN,0.500000,HVAC,NaN,NaN,NaN
19998,WO598803,2019-05-10 17:27:51,2019-05-13 12:27:51,11.0,NaN,0.500000,Building Maintenance,NaN,NaN,NaN


### Status

In [62]:
status_list = ['Cancel', 'Inprogress', 'Complete', 'Onhold']
status_weight = [0.06, 0.15, 0.74, 0.05]

In [63]:
sum(status_weight)

1.0

In [64]:
df['Status'] = choice(status_list, 20000, p = status_weight)

In [65]:
df['Status'].value_counts()

Complete      14800
Inprogress     3030
Cancel         1185
Onhold          985
Name: Status, dtype: int64

In [66]:
df.head(10)

,OrderID,Startdate,Enddate,Priority,Status,ActLaborHrs,Group,SubGroup,Responsedays,Responsehours
0,WO156580,2019-04-05 09:31:17,2019-04-05 16:31:17,2.0,Complete,1.000000,HVAC,NaN,NaN,NaN
1,WO372550,2018-01-05 13:14:41,2018-01-09 14:14:41,11.0,Complete,9.039537,Engineering,NaN,NaN,NaN
2,WO293904,2019-01-31 14:49:44,2019-02-01 09:49:44,2.0,Complete,9.039537,CallCenter,NaN,NaN,NaN
3,WO220975,2019-06-04 16:05:03,2019-06-07 11:05:03,11.0,Complete,1.000000,Plumber,NaN,NaN,NaN
4,WO352047,2019-08-28 15:55:37,2019-08-28 16:55:37,11.0,Complete,0.500000,Engineering,NaN,NaN,NaN
5,WO204383,2019-01-10 10:33:59,2019-01-10 11:33:59,11.0,Complete,1.000000,Cleaning,NaN,NaN,NaN
6,WO348957,2019-10-29 14:30:37,2019-10-30 09:30:37,11.0,Complete,9.039537,Cleaning,NaN,NaN,NaN
7,WO389943,2019-05-16 17:01:59,2019-05-20 12:01:59,11.0,Complete,0.500000,Building Maintenance,NaN,NaN,NaN
8,WO506187,2019-03-15 13:55:07,2019-03-18 14:55:07,11.0,Complete,0.500000,Building Maintenance,NaN,NaN,NaN
9,WO415139,2018-07-02 15:55:03,2018-07-06 10:55:03,2.0,Complete,9.039537,Locksmith,NaN,NaN,NaN


In [ ]:
#df.to_csv('fake_data1.csv')